In [2]:
import h5py
from os import listdir
from os.path import isfile, join, isdir
import numpy as np

In [3]:
mypath = "Data"
onlydirs = [f for f in listdir(mypath) if isdir(join(mypath, f))]
h5filelist = []
samplelist = {}

In [4]:
onlydirs

['AIS TM349_1 x60-1MaxPrj Series TM349_1 x60-1MaxPrj.tif',
 'AIS TM349_1 x60-2MaxPrj Series TM349_1 x60-2MaxPrj.tif',
 'AIS TM349_3 x60-1MaxPrj Series TM349_3 x60-1MaxPrj.tif',
 'AIS TM349_3 x60-2MaxPrj Series TM349_3 x60-2MaxPrj.tif',
 'AIS TM349_7 x60-2MaxPrj Series TM349_7 x60-2MaxPrj.tif',
 'AIS TM349_7 x60-3MaxPrj Series TM349_7 x60-3MaxPrj.tif',
 'AIS TM349_7 x60-4MaxPrj Series TM349_7 x60-4MaxPrj.tif']

In [5]:
for dirpath in onlydirs:
    aispath = join(mypath,dirpath)
    files = [f for f in listdir(aispath) if isfile(join(aispath, f))]
    h5files = [join(aispath,f) for f in files if f.endswith(".h5")]
    h5filelist += h5files

In [6]:
h5filelist

['Data\\AIS TM349_1 x60-1MaxPrj Series TM349_1 x60-1MaxPrj.tif\\TM349_1 x60-1MaxPrj.h5',
 'Data\\AIS TM349_1 x60-2MaxPrj Series TM349_1 x60-2MaxPrj.tif\\TM349_1 x60-2MaxPrj.h5',
 'Data\\AIS TM349_3 x60-1MaxPrj Series TM349_3 x60-1MaxPrj.tif\\TM349_3 x60-1MaxPrj.h5',
 'Data\\AIS TM349_3 x60-2MaxPrj Series TM349_3 x60-2MaxPrj.tif\\TM349_3 x60-2MaxPrj.h5',
 'Data\\AIS TM349_7 x60-2MaxPrj Series TM349_7 x60-2MaxPrj.tif\\TM349_7 x60-2MaxPrj.h5',
 'Data\\AIS TM349_7 x60-3MaxPrj Series TM349_7 x60-3MaxPrj.tif\\TM349_7 x60-3MaxPrj.h5',
 'Data\\AIS TM349_7 x60-4MaxPrj Series TM349_7 x60-4MaxPrj.tif\\TM349_7 x60-4MaxPrj.h5']

In [8]:
for h5file in h5filelist:
    if h5py.is_hdf5(h5file):
        lala = h5py.File(h5file,"r")
        
        try:
            rois = [key for key in lala["Data"].keys()]
            lengths = []
            for roi in rois:
                length = lala["Data/"+roi+"/Physical"].attrs["AISPhysicalLength"]
                #length = lala["Data/"+roi+"/Pixeldata"].attrs["AISend"] - lala["Data/"+roi+"/Pixeldata"].attrs["AISstart"]
                lengths.append(length)
        
            lengths = np.array(lengths)
            seriesname = lala.attrs["seriesname"].split("_")[0]
            samplelist[seriesname] = np.append(samplelist.get(seriesname,np.array([])),lengths)
            print(lala.attrs["seriesname"] + "    " +str(lengths.mean()) + "   " + str(np.std(lengths)))
        except KeyError as e :
            print("File {0} doesnt hava data Object: {1}".format(h5file,e))
            pass
        
    else:
        print("File " + h5file + " doesn't conform with h5 standards")
        pass

TM349_1 x60-1MaxPrj.tif    32.8410328778   11.2877844812
TM349_1 x60-2MaxPrj.tif    42.1842326777   14.5757373694
TM349_3 x60-1MaxPrj.tif    40.8233375421   10.9488648385
File Data\AIS TM349_3 x60-2MaxPrj Series TM349_3 x60-2MaxPrj.tif\TM349_3 x60-2MaxPrj.h5 doesnt hava data Object: "Unable to open object (Object 'data' doesn't exist)"
File Data\AIS TM349_7 x60-2MaxPrj Series TM349_7 x60-2MaxPrj.tif\TM349_7 x60-2MaxPrj.h5 doesnt hava data Object: "Unable to open object (Object 'data' doesn't exist)"
TM349_7 x60-3MaxPrj.tif    40.5393781566   9.60877796342
File Data\AIS TM349_7 x60-4MaxPrj Series TM349_7 x60-4MaxPrj.tif\TM349_7 x60-4MaxPrj.h5 doesnt hava data Object: "Unable to open object (Object 'data' doesn't exist)"


In [9]:
for key,value in samplelist.items():
    print("Sample: {0}  Mean: {1} Std: {2} AIS: {3}".format(key,value.mean(),np.std(value),len(value)))

Sample: TM349  Mean: 39.53063241106719 Std: 11.88780422120778 AIS: 184


In [22]:
from scipy.stats import ttest_ind, ttest_rel

t, p = ttest_ind(samplelist["TM349"],samplelist["TM340"])

In [24]:
pvalue = "{0:.10f}".format(p)
tvalue = "{0:.10f}".format(t)

In [14]:
samplelist["TM340"].size

40

In [25]:
print("T-Wert: {0} P-Wert: {1}".format(tvalue,pvalue))

T-Wert: 4.8303691249 P-Wert: 0.0000021309
